In [7]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from PIL import Image

# Function to extract features using Img2Vec
def extract_features(image_paths, class_name):
    img2vec = Img2Vec()
    features = []
    used_count = 0
    skipped_count = 0
    for img_path in image_paths:
        try:
            img = Image.open(img_path)
            vec = img2vec.get_vec(img)
            features.append(vec)
            used_count += 1
        except (Image.UnidentifiedImageError, OSError):
            print("Skipping image:", img_path)
            skipped_count += 1
    print(f"For class {class_name}: Used {used_count} images, Skipped {skipped_count} images")
    return np.array(features)

# Define your dataset directory
dataset_dir = "Dataset"

# Get list of class folders
class_folders = [folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))]

# Prepare data and labels
data = []
labels = []
image_count = 0
for i, folder in enumerate(class_folders):
    class_dir = os.path.join(dataset_dir, folder)
    image_files = [os.path.join(class_dir, file) for file in os.listdir(class_dir) if file.endswith(('.jpg', '.jpeg', '.png'))]
    image_count += len(image_files)
    features = extract_features(image_files, folder)
    data.extend(features)
    labels.extend([i]*len(features))

print("Total number of images in the dataset:", image_count)

# Convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Number of images in X_train:", len(X_train))
print("Number of images in X_test:", len(X_test))

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get classification report
class_names = [folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))]
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))


For class Ballet: Used 800 images, Skipped 0 images
For class Dragon Dance: Used 800 images, Skipped 0 images
Skipping image: Dataset\Fandango\100.jpg
Skipping image: Dataset\Fandango\101.jpg
Skipping image: Dataset\Fandango\103.jpg
Skipping image: Dataset\Fandango\104.jpg
Skipping image: Dataset\Fandango\105.jpg
Skipping image: Dataset\Fandango\107.jpg
Skipping image: Dataset\Fandango\108.jpg
Skipping image: Dataset\Fandango\110.jpg
Skipping image: Dataset\Fandango\111.jpg
Skipping image: Dataset\Fandango\112.jpg
Skipping image: Dataset\Fandango\113.jpg
Skipping image: Dataset\Fandango\114.jpg
Skipping image: Dataset\Fandango\116.jpg
Skipping image: Dataset\Fandango\117.jpg
Skipping image: Dataset\Fandango\118.jpg
Skipping image: Dataset\Fandango\119.jpg
Skipping image: Dataset\Fandango\120.jpg
Skipping image: Dataset\Fandango\129.jpg
Skipping image: Dataset\Fandango\130.jpg
Skipping image: Dataset\Fandango\132.jpg
Skipping image: Dataset\Fandango\133.jpg
Skipping image: Dataset\Fanda

Skipping image: Dataset\Fandango\510.jpg
Skipping image: Dataset\Fandango\511.jpg
Skipping image: Dataset\Fandango\514.jpg
Skipping image: Dataset\Fandango\515.jpg
Skipping image: Dataset\Fandango\516.jpg
Skipping image: Dataset\Fandango\517.jpg
Skipping image: Dataset\Fandango\519.jpg
Skipping image: Dataset\Fandango\520.jpg
Skipping image: Dataset\Fandango\521.jpg
Skipping image: Dataset\Fandango\522.jpg
Skipping image: Dataset\Fandango\523.jpg
Skipping image: Dataset\Fandango\526.jpg
Skipping image: Dataset\Fandango\528.jpg
Skipping image: Dataset\Fandango\532.jpg
Skipping image: Dataset\Fandango\533.jpg
Skipping image: Dataset\Fandango\535.jpg
Skipping image: Dataset\Fandango\537.jpg
Skipping image: Dataset\Fandango\538.jpg
Skipping image: Dataset\Fandango\540.jpg
Skipping image: Dataset\Fandango\541.jpg
Skipping image: Dataset\Fandango\542.jpg
Skipping image: Dataset\Fandango\546.jpg
Skipping image: Dataset\Fandango\547.jpg
Skipping image: Dataset\Fandango\548.jpg
Skipping image: 

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [11]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

# Function to extract features using a pre-trained model
def extract_features(image_paths):
    # Load a pre-trained ResNet model
    resnet = models.resnet18(pretrained=True)
    # Remove the classification layer (the last layer) to get feature vectors
    resnet_feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])
    # Set the model to evaluation mode
    resnet_feature_extractor.eval()

    features = []
    for img_path in image_paths:
        try:
            # Open and preprocess the image
            img = Image.open(img_path).convert('RGB')
            preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            img_tensor = preprocess(img)
            img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension

            # Extract features
            with torch.no_grad():
                feature_vector = resnet_feature_extractor(img_tensor)
            features.append(feature_vector.squeeze().numpy())
        except (OSError, Exception) as e:
            print("Skipping image:", img_path)
            print("Error:", e)

    return features

# Define your dataset directory
dataset_dir = "Dataset"

# Get list of class folders
class_folders = [folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))]

# Prepare data and labels
data = []
labels = []
image_count = 0
for i, folder in enumerate(class_folders):
    class_dir = os.path.join(dataset_dir, folder)
    image_files = [os.path.join(class_dir, file) for file in os.listdir(class_dir) if file.endswith(('.jpg', '.jpeg', '.png'))]
    image_count += len(image_files)
    features = extract_features(image_files)
    data.extend(features)
    labels.extend([i]*len(features))

print("Total number of images in the dataset:", image_count)
print("Number of features extracted:", len(data))

data = np.array(data)
labels = np.array(labels)

# Split data into train and test sets (example)
# You can adjust this according to your needs
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train your model or perform further processing here
# Example:
# model.fit(X_train, y_train)


Skipping image: Dataset\Fandango\100.jpg
Error: cannot identify image file 'Dataset\\Fandango\\100.jpg'
Skipping image: Dataset\Fandango\101.jpg
Error: cannot identify image file 'Dataset\\Fandango\\101.jpg'
Skipping image: Dataset\Fandango\103.jpg
Error: cannot identify image file 'Dataset\\Fandango\\103.jpg'
Skipping image: Dataset\Fandango\104.jpg
Error: cannot identify image file 'Dataset\\Fandango\\104.jpg'
Skipping image: Dataset\Fandango\105.jpg
Error: cannot identify image file 'Dataset\\Fandango\\105.jpg'
Skipping image: Dataset\Fandango\107.jpg
Error: cannot identify image file 'Dataset\\Fandango\\107.jpg'
Skipping image: Dataset\Fandango\108.jpg
Error: cannot identify image file 'Dataset\\Fandango\\108.jpg'
Skipping image: Dataset\Fandango\110.jpg
Error: cannot identify image file 'Dataset\\Fandango\\110.jpg'
Skipping image: Dataset\Fandango\111.jpg
Error: cannot identify image file 'Dataset\\Fandango\\111.jpg'
Skipping image: Dataset\Fandango\112.jpg
Error: cannot identify 

Skipping image: Dataset\Fandango\274.jpg
Error: cannot identify image file 'Dataset\\Fandango\\274.jpg'
Skipping image: Dataset\Fandango\28.jpg
Error: cannot identify image file 'Dataset\\Fandango\\28.jpg'
Skipping image: Dataset\Fandango\280.jpg
Error: cannot identify image file 'Dataset\\Fandango\\280.jpg'
Skipping image: Dataset\Fandango\283.jpg
Error: cannot identify image file 'Dataset\\Fandango\\283.jpg'
Skipping image: Dataset\Fandango\285.jpg
Error: cannot identify image file 'Dataset\\Fandango\\285.jpg'
Skipping image: Dataset\Fandango\286.jpg
Error: cannot identify image file 'Dataset\\Fandango\\286.jpg'
Skipping image: Dataset\Fandango\287.jpg
Error: cannot identify image file 'Dataset\\Fandango\\287.jpg'
Skipping image: Dataset\Fandango\288.jpg
Error: cannot identify image file 'Dataset\\Fandango\\288.jpg'
Skipping image: Dataset\Fandango\29.jpg
Error: cannot identify image file 'Dataset\\Fandango\\29.jpg'
Skipping image: Dataset\Fandango\295.jpg
Error: cannot identify imag

Skipping image: Dataset\Fandango\427.jpg
Error: cannot identify image file 'Dataset\\Fandango\\427.jpg'
Skipping image: Dataset\Fandango\43.jpg
Error: cannot identify image file 'Dataset\\Fandango\\43.jpg'
Skipping image: Dataset\Fandango\430.jpg
Error: cannot identify image file 'Dataset\\Fandango\\430.jpg'
Skipping image: Dataset\Fandango\431.jpg
Error: cannot identify image file 'Dataset\\Fandango\\431.jpg'
Skipping image: Dataset\Fandango\433.jpg
Error: cannot identify image file 'Dataset\\Fandango\\433.jpg'
Skipping image: Dataset\Fandango\434.jpg
Error: cannot identify image file 'Dataset\\Fandango\\434.jpg'
Skipping image: Dataset\Fandango\438.jpg
Error: cannot identify image file 'Dataset\\Fandango\\438.jpg'
Skipping image: Dataset\Fandango\44.jpg
Error: cannot identify image file 'Dataset\\Fandango\\44.jpg'
Skipping image: Dataset\Fandango\440.jpg
Error: cannot identify image file 'Dataset\\Fandango\\440.jpg'
Skipping image: Dataset\Fandango\441.jpg
Error: cannot identify imag

Skipping image: Dataset\Fandango\608.jpg
Error: cannot identify image file 'Dataset\\Fandango\\608.jpg'
Skipping image: Dataset\Fandango\61.jpg
Error: cannot identify image file 'Dataset\\Fandango\\61.jpg'
Skipping image: Dataset\Fandango\610.jpg
Error: cannot identify image file 'Dataset\\Fandango\\610.jpg'
Skipping image: Dataset\Fandango\611.jpg
Error: cannot identify image file 'Dataset\\Fandango\\611.jpg'
Skipping image: Dataset\Fandango\617.jpg
Error: cannot identify image file 'Dataset\\Fandango\\617.jpg'
Skipping image: Dataset\Fandango\619.jpg
Error: cannot identify image file 'Dataset\\Fandango\\619.jpg'
Skipping image: Dataset\Fandango\620.jpg
Error: cannot identify image file 'Dataset\\Fandango\\620.jpg'
Skipping image: Dataset\Fandango\622.jpg
Error: cannot identify image file 'Dataset\\Fandango\\622.jpg'
Skipping image: Dataset\Fandango\624.jpg
Error: cannot identify image file 'Dataset\\Fandango\\624.jpg'
Skipping image: Dataset\Fandango\627.jpg
Error: cannot identify im

Skipping image: Dataset\Fandango\786.jpg
Error: cannot identify image file 'Dataset\\Fandango\\786.jpg'
Skipping image: Dataset\Fandango\789.jpg
Error: cannot identify image file 'Dataset\\Fandango\\789.jpg'
Skipping image: Dataset\Fandango\79.jpg
Error: cannot identify image file 'Dataset\\Fandango\\79.jpg'
Skipping image: Dataset\Fandango\791.jpg
Error: cannot identify image file 'Dataset\\Fandango\\791.jpg'
Skipping image: Dataset\Fandango\793.jpg
Error: cannot identify image file 'Dataset\\Fandango\\793.jpg'
Skipping image: Dataset\Fandango\800.jpg
Error: cannot identify image file 'Dataset\\Fandango\\800.jpg'
Skipping image: Dataset\Fandango\82.jpg
Error: cannot identify image file 'Dataset\\Fandango\\82.jpg'
Skipping image: Dataset\Fandango\87.jpg
Error: cannot identify image file 'Dataset\\Fandango\\87.jpg'
Skipping image: Dataset\Fandango\89.jpg
Error: cannot identify image file 'Dataset\\Fandango\\89.jpg'
Skipping image: Dataset\Fandango\91.jpg
Error: cannot identify image fil

Skipping image: Dataset\Manipuri\213.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\213.jpg'
Skipping image: Dataset\Manipuri\215.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\215.jpg'
Skipping image: Dataset\Manipuri\217.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\217.jpg'
Skipping image: Dataset\Manipuri\22.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\22.jpg'
Skipping image: Dataset\Manipuri\222.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\222.jpg'
Skipping image: Dataset\Manipuri\223.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\223.jpg'
Skipping image: Dataset\Manipuri\224.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\224.jpg'
Skipping image: Dataset\Manipuri\226.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\226.jpg'
Skipping image: Dataset\Manipuri\227.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\227.jpg'
Skipping image: Dataset\Manipuri\23.jpg
Error: cannot identify ima

Skipping image: Dataset\Manipuri\340.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\340.jpg'
Skipping image: Dataset\Manipuri\345.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\345.jpg'
Skipping image: Dataset\Manipuri\346.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\346.jpg'
Skipping image: Dataset\Manipuri\347.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\347.jpg'
Skipping image: Dataset\Manipuri\348.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\348.jpg'
Skipping image: Dataset\Manipuri\349.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\349.jpg'
Skipping image: Dataset\Manipuri\35.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\35.jpg'
Skipping image: Dataset\Manipuri\351.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\351.jpg'
Skipping image: Dataset\Manipuri\353.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\353.jpg'
Skipping image: Dataset\Manipuri\355.jpg
Error: cannot identify im

Skipping image: Dataset\Manipuri\456.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\456.jpg'
Skipping image: Dataset\Manipuri\46.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\46.jpg'
Skipping image: Dataset\Manipuri\461.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\461.jpg'
Skipping image: Dataset\Manipuri\464.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\464.jpg'
Skipping image: Dataset\Manipuri\465.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\465.jpg'
Skipping image: Dataset\Manipuri\466.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\466.jpg'
Skipping image: Dataset\Manipuri\468.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\468.jpg'
Skipping image: Dataset\Manipuri\469.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\469.jpg'
Skipping image: Dataset\Manipuri\470.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\470.jpg'
Skipping image: Dataset\Manipuri\471.jpg
Error: cannot identify im

Skipping image: Dataset\Manipuri\579.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\579.jpg'
Skipping image: Dataset\Manipuri\583.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\583.jpg'
Skipping image: Dataset\Manipuri\584.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\584.jpg'
Skipping image: Dataset\Manipuri\587.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\587.jpg'
Skipping image: Dataset\Manipuri\589.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\589.jpg'
Skipping image: Dataset\Manipuri\590.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\590.jpg'
Skipping image: Dataset\Manipuri\595.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\595.jpg'
Skipping image: Dataset\Manipuri\596.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\596.jpg'
Skipping image: Dataset\Manipuri\597.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\597.jpg'
Skipping image: Dataset\Manipuri\599.jpg
Error: cannot identify 

Skipping image: Dataset\Manipuri\711.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\711.jpg'
Skipping image: Dataset\Manipuri\712.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\712.jpg'
Skipping image: Dataset\Manipuri\713.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\713.jpg'
Skipping image: Dataset\Manipuri\714.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\714.jpg'
Skipping image: Dataset\Manipuri\715.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\715.jpg'
Skipping image: Dataset\Manipuri\718.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\718.jpg'
Skipping image: Dataset\Manipuri\719.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\719.jpg'
Skipping image: Dataset\Manipuri\72.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\72.jpg'
Skipping image: Dataset\Manipuri\720.jpg
Error: cannot identify image file 'Dataset\\Manipuri\\720.jpg'
Skipping image: Dataset\Manipuri\721.jpg
Error: cannot identify im

In [12]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Number of images in X_train:", len(X_train))
print("Number of images in X_test:", len(X_test))

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get classification report
class_names = [folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))]
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))


Number of images in X_train: 4475
Number of images in X_test: 1119
Accuracy: 0.966041108132261
Classification Report:
              precision    recall  f1-score   support

      Ballet       0.96      0.97      0.97       181
Dragon Dance       0.97      1.00      0.99       168
    Fandango       0.99      0.89      0.94        83
      Kabuki       0.96      0.98      0.97       161
   Kathakali       0.99      0.99      0.99       149
    Manipuri       1.00      0.87      0.93        60
      Odissi       0.96      0.96      0.96       164
       Waltz       0.93      0.97      0.95       153

    accuracy                           0.97      1119
   macro avg       0.97      0.95      0.96      1119
weighted avg       0.97      0.97      0.97      1119



In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=8)

# Train the classifier
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.998212689901698


In [18]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

In [19]:
# Support Vector Machine (SVM) Classifier
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

# Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)


# CatBoost Classifier
catboost = CatBoostClassifier()
catboost.fit(X_train, y_train)
y_pred_catboost = catboost.predict(X_test)
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print("CatBoost Accuracy:", accuracy_catboost)

# AdaBoost Classifier
adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)
y_pred_adaboost = adaboost.predict(X_test)
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print("AdaBoost Accuracy:", accuracy_adaboost)

# XGBoost Classifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)

# Naïve Bayes Classifier
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred_naive_bayes = naive_bayes.predict(X_test)
accuracy_naive_bayes = accuracy_score(y_test, y_pred_naive_bayes)
print("Naïve Bayes Accuracy:", accuracy_naive_bayes)

SVM Accuracy: 0.9973190348525469
Decision Tree Accuracy: 0.6550491510277033
Learning rate set to 0.085411
0:	learn: 1.9761462	total: 549ms	remaining: 9m 8s
1:	learn: 1.8811307	total: 850ms	remaining: 7m 4s
2:	learn: 1.8008839	total: 1.18s	remaining: 6m 30s
3:	learn: 1.7350795	total: 1.48s	remaining: 6m 9s
4:	learn: 1.6727416	total: 1.8s	remaining: 5m 59s
5:	learn: 1.6177364	total: 2.13s	remaining: 5m 52s
6:	learn: 1.5657473	total: 2.44s	remaining: 5m 46s
7:	learn: 1.5176048	total: 2.76s	remaining: 5m 42s
8:	learn: 1.4676328	total: 3.06s	remaining: 5m 37s
9:	learn: 1.4279343	total: 3.37s	remaining: 5m 33s
10:	learn: 1.3857048	total: 3.69s	remaining: 5m 31s
11:	learn: 1.3423208	total: 4s	remaining: 5m 28s
12:	learn: 1.3068799	total: 4.31s	remaining: 5m 27s
13:	learn: 1.2742987	total: 4.62s	remaining: 5m 25s
14:	learn: 1.2459334	total: 4.92s	remaining: 5m 23s
15:	learn: 1.2133402	total: 5.24s	remaining: 5m 22s
16:	learn: 1.1794459	total: 5.55s	remaining: 5m 21s
17:	learn: 1.1515103	total:

156:	learn: 0.1942050	total: 50.6s	remaining: 4m 31s
157:	learn: 0.1924498	total: 50.9s	remaining: 4m 31s
158:	learn: 0.1907609	total: 51.2s	remaining: 4m 30s
159:	learn: 0.1896909	total: 51.5s	remaining: 4m 30s
160:	learn: 0.1881091	total: 51.9s	remaining: 4m 30s
161:	learn: 0.1867088	total: 52.2s	remaining: 4m 30s
162:	learn: 0.1854863	total: 52.5s	remaining: 4m 29s
163:	learn: 0.1843705	total: 52.9s	remaining: 4m 29s
164:	learn: 0.1823076	total: 53.2s	remaining: 4m 29s
165:	learn: 0.1804955	total: 53.5s	remaining: 4m 28s
166:	learn: 0.1788690	total: 53.8s	remaining: 4m 28s
167:	learn: 0.1777180	total: 54.2s	remaining: 4m 28s
168:	learn: 0.1759463	total: 54.5s	remaining: 4m 27s
169:	learn: 0.1748912	total: 54.8s	remaining: 4m 27s
170:	learn: 0.1736652	total: 55.1s	remaining: 4m 27s
171:	learn: 0.1721801	total: 55.5s	remaining: 4m 27s
172:	learn: 0.1706310	total: 55.8s	remaining: 4m 26s
173:	learn: 0.1691865	total: 56.2s	remaining: 4m 26s
174:	learn: 0.1681907	total: 56.5s	remaining: 

310:	learn: 0.0805035	total: 1m 41s	remaining: 3m 45s
311:	learn: 0.0800402	total: 1m 41s	remaining: 3m 44s
312:	learn: 0.0797710	total: 1m 42s	remaining: 3m 44s
313:	learn: 0.0794668	total: 1m 42s	remaining: 3m 44s
314:	learn: 0.0791916	total: 1m 42s	remaining: 3m 43s
315:	learn: 0.0790322	total: 1m 43s	remaining: 3m 43s
316:	learn: 0.0785813	total: 1m 43s	remaining: 3m 43s
317:	learn: 0.0781819	total: 1m 43s	remaining: 3m 42s
318:	learn: 0.0778039	total: 1m 44s	remaining: 3m 42s
319:	learn: 0.0773678	total: 1m 44s	remaining: 3m 42s
320:	learn: 0.0771723	total: 1m 44s	remaining: 3m 41s
321:	learn: 0.0768468	total: 1m 45s	remaining: 3m 41s
322:	learn: 0.0764219	total: 1m 45s	remaining: 3m 41s
323:	learn: 0.0759953	total: 1m 45s	remaining: 3m 41s
324:	learn: 0.0754925	total: 1m 46s	remaining: 3m 40s
325:	learn: 0.0752106	total: 1m 46s	remaining: 3m 40s
326:	learn: 0.0749059	total: 1m 46s	remaining: 3m 40s
327:	learn: 0.0746203	total: 1m 47s	remaining: 3m 39s
328:	learn: 0.0743617	total:

464:	learn: 0.0467334	total: 2m 34s	remaining: 2m 58s
465:	learn: 0.0466310	total: 2m 35s	remaining: 2m 57s
466:	learn: 0.0464602	total: 2m 35s	remaining: 2m 57s
467:	learn: 0.0463066	total: 2m 35s	remaining: 2m 57s
468:	learn: 0.0460309	total: 2m 36s	remaining: 2m 56s
469:	learn: 0.0459129	total: 2m 36s	remaining: 2m 56s
470:	learn: 0.0457419	total: 2m 36s	remaining: 2m 56s
471:	learn: 0.0455833	total: 2m 37s	remaining: 2m 55s
472:	learn: 0.0454076	total: 2m 37s	remaining: 2m 55s
473:	learn: 0.0452912	total: 2m 37s	remaining: 2m 55s
474:	learn: 0.0451889	total: 2m 38s	remaining: 2m 54s
475:	learn: 0.0450369	total: 2m 38s	remaining: 2m 54s
476:	learn: 0.0448385	total: 2m 38s	remaining: 2m 54s
477:	learn: 0.0446954	total: 2m 39s	remaining: 2m 53s
478:	learn: 0.0445353	total: 2m 39s	remaining: 2m 53s
479:	learn: 0.0443073	total: 2m 39s	remaining: 2m 53s
480:	learn: 0.0441997	total: 2m 40s	remaining: 2m 52s
481:	learn: 0.0440886	total: 2m 40s	remaining: 2m 52s
482:	learn: 0.0439429	total:

617:	learn: 0.0310439	total: 3m 27s	remaining: 2m 8s
618:	learn: 0.0309842	total: 3m 27s	remaining: 2m 7s
619:	learn: 0.0309342	total: 3m 27s	remaining: 2m 7s
620:	learn: 0.0308568	total: 3m 28s	remaining: 2m 7s
621:	learn: 0.0307543	total: 3m 28s	remaining: 2m 6s
622:	learn: 0.0306840	total: 3m 28s	remaining: 2m 6s
623:	learn: 0.0306018	total: 3m 29s	remaining: 2m 6s
624:	learn: 0.0305522	total: 3m 29s	remaining: 2m 5s
625:	learn: 0.0304902	total: 3m 29s	remaining: 2m 5s
626:	learn: 0.0304119	total: 3m 30s	remaining: 2m 5s
627:	learn: 0.0303413	total: 3m 30s	remaining: 2m 4s
628:	learn: 0.0303005	total: 3m 30s	remaining: 2m 4s
629:	learn: 0.0302660	total: 3m 31s	remaining: 2m 4s
630:	learn: 0.0302095	total: 3m 31s	remaining: 2m 3s
631:	learn: 0.0301375	total: 3m 31s	remaining: 2m 3s
632:	learn: 0.0300350	total: 3m 32s	remaining: 2m 3s
633:	learn: 0.0299775	total: 3m 32s	remaining: 2m 2s
634:	learn: 0.0299248	total: 3m 33s	remaining: 2m 2s
635:	learn: 0.0298217	total: 3m 33s	remaining:

771:	learn: 0.0225571	total: 4m 21s	remaining: 1m 17s
772:	learn: 0.0225187	total: 4m 21s	remaining: 1m 16s
773:	learn: 0.0224563	total: 4m 22s	remaining: 1m 16s
774:	learn: 0.0224319	total: 4m 22s	remaining: 1m 16s
775:	learn: 0.0223873	total: 4m 22s	remaining: 1m 15s
776:	learn: 0.0223423	total: 4m 23s	remaining: 1m 15s
777:	learn: 0.0222993	total: 4m 23s	remaining: 1m 15s
778:	learn: 0.0222160	total: 4m 23s	remaining: 1m 14s
779:	learn: 0.0221543	total: 4m 24s	remaining: 1m 14s
780:	learn: 0.0220859	total: 4m 24s	remaining: 1m 14s
781:	learn: 0.0220539	total: 4m 24s	remaining: 1m 13s
782:	learn: 0.0220366	total: 4m 25s	remaining: 1m 13s
783:	learn: 0.0219891	total: 4m 25s	remaining: 1m 13s
784:	learn: 0.0219503	total: 4m 25s	remaining: 1m 12s
785:	learn: 0.0219184	total: 4m 26s	remaining: 1m 12s
786:	learn: 0.0218910	total: 4m 26s	remaining: 1m 12s
787:	learn: 0.0218647	total: 4m 26s	remaining: 1m 11s
788:	learn: 0.0218237	total: 4m 27s	remaining: 1m 11s
789:	learn: 0.0217766	total:

927:	learn: 0.0170533	total: 5m 19s	remaining: 24.8s
928:	learn: 0.0170375	total: 5m 19s	remaining: 24.4s
929:	learn: 0.0170107	total: 5m 19s	remaining: 24.1s
930:	learn: 0.0169678	total: 5m 20s	remaining: 23.7s
931:	learn: 0.0169521	total: 5m 20s	remaining: 23.4s
932:	learn: 0.0169225	total: 5m 21s	remaining: 23.1s
933:	learn: 0.0168947	total: 5m 21s	remaining: 22.7s
934:	learn: 0.0168551	total: 5m 21s	remaining: 22.4s
935:	learn: 0.0168362	total: 5m 22s	remaining: 22s
936:	learn: 0.0168088	total: 5m 22s	remaining: 21.7s
937:	learn: 0.0167827	total: 5m 23s	remaining: 21.4s
938:	learn: 0.0167613	total: 5m 23s	remaining: 21s
939:	learn: 0.0167351	total: 5m 23s	remaining: 20.7s
940:	learn: 0.0167171	total: 5m 24s	remaining: 20.3s
941:	learn: 0.0166842	total: 5m 24s	remaining: 20s
942:	learn: 0.0166587	total: 5m 25s	remaining: 19.6s
943:	learn: 0.0166273	total: 5m 25s	remaining: 19.3s
944:	learn: 0.0166112	total: 5m 25s	remaining: 19s
945:	learn: 0.0165708	total: 5m 26s	remaining: 18.6s
9